# `Tokenizer` Tutorial
   
This notebook is to show examples of how to use the `tokenizer` to divide texts into "tokens". A token is a countable entity, which serves as the basis for computational analysis. Most of the time, tokens will correspond to words, but they may also be characters, punctuation marks, or even spaces.

Once a text is tokenised, it is possible to generate a list of unique token forms in the text, generally known as "types" or "terms" (we use the latter here). The frequency with which individual terms occur in a text is often revealing about the style, meaning, or authorship of the text.

## About `Tokenizer`

It is possible to produce a list of word tokens for an English text by dividing words on every white space in the text using a tool like Python's `split()` function. However, this will not work 100% of the time and may work far less well for some other languages. The Lexos `Tokenizer` takes advantage of "language models" to automate the process. Language models can implement both rule-based and probabilistic strategies for separating document strings into tokens. Because they have built-in procedures appropriate to specific languages, language models can often do a better job of tokenisation than the approach used in the Lexos app.

There are some trade-offs to using language models. Because the algorithm does more than split strings, processing times can be greater. In addition, tokenisation is no longer (explicitly) language agnostic. A language model is "opinionated" and it may overfit the data. Likewise, if no language model exists for the language being tokenised, the results may not be satisfactory. The Lexos strategy for handling this situation is described below.

Behind the scenes, Lexos uses the Python spaCy library to tokenise texts. The result is called a spaCy doc (short for document). Each spaCy doc has a `text` attribute (which is the original text) and a list of tokens, each with their own attributes. spaCy has a lot of built-in attributes: things like `is_punct` (whether or not the token is a punctuation mark) or `is_digit` (whether or not the token is a digit). You can see a <a href="https://spacy.io/api/token#attributes" target="_blank">complete list</a> in spaCy documentation. Depending on how the language model has been trained, you may get more or less information. For instance, spaCy's "en_web_core_sm" English-language model tags the part of speech of every word. You can load this (or another) model into the Lexos `Tokenizer` if you want. However, Lexos does not assume that you are working in English, so the default model is spaCy's "xx_sent_ud_sm" multilanguage model, which does a good job finding sentence and token boundaries for a wide variety of languages but does not provide as much information as some of the other models.

You can see whether spaCy has a model for your language on the <a href="https://spacy.io/models" target="_blank">spaCy models</a> webpage. You can load any of these models into the Lexos `Tokenizer`, but you will need to download the model first by copying the code provided on that page.

This is probably enough information to get started, so let's get to work.

## Load Some Data

We'll start by loading some data using the `Loader` module. We're going to take the first 1245 characters of Jane Austen's _Pride and Prejudice_? Why 1245? Because it's a relatively small passage that we can process quickly (the full novel would take much longer) and because character 1245 comes at the end of a sentence.

In [ ]:
from lexos.io.smart import Loader

loader = Loader()
loader.load("../test_data/txt/Austen_Pride.txt")
text = loader.texts[0].strip()[0:1245]
text


## Import `Tokenizer`

Now we'll attempt to tokenise this text. We'll start by importing the `tokenizer` module.

In [ ]:
from lexos import tokenizer

## Making a Doc

`Tokenizer` has a function called `make_doc()` to which we can feed our text. Remember that by default, `Tokenizer` uses spaCy's "xx_sent_ud_sm" multilanguage model.

We can view the doc's original text by referencing `doc.text`, or we can print out the text of each token in the document using a `for` loop. In the example below, we will only print out snippets of the text and tokens. We enclose tokens in angle brackets for greater visibility and to show that some tokens are line breaks or punctuation marks (something we may have to deal with by scrubbing our text first or by filtering our tokens later).

In [ ]:
doc = tokenizer.make_doc(text)
print("\nText:")
print("=====")

print(doc.text[0:100])

print("\nTokens:")
print("=======")
for token in doc[0:60]:
    print(f"<{token.text}>")

## Specifying a Language Model

You can specify a language model with the `model` parameter. In the example below, we load the "en_core_web_sm" model. Notice how much longer it takes to tokenise. But notice the information that we get out of the model (in this example, we are printing out the part of speech for each token).

In [ ]:
doc = tokenizer.make_doc(text, model="en_core_web_sm")
for token in doc[0:60]:
    print(f"<{token.text}>: {token.pos_}")

A more concise (and faster) way of generating a token list is with a Python "list comprehension", as demonstrated in the example below. The example further demonstrates how to dispay the results in a pandas DataFrame. 

In [ ]:
# Import pandas
import pandas as pd

# Use a list comprehension to generate a list of token dicts
tokens = [{"Token": token.text, "POS": token.pos_} for token in doc]

df = pd.DataFrame(tokens)
df.head(10)

## Tokenising Multiple Texts

You can use the `Tokenizer.make_docs()` function to make process multiple texts at once. In this example, we are just going to cut our text roughly in half to make two separate texts and then convert them to spaCy docs. Although we are going to feed our texts to `make_docs()` in a list in this example, recall that `loader.texts` _is_ a list, so a common procedure would be to call `make_docs(loader.texts)`.

In [ ]:
text1 = text[0:565]
text2 = text[565:1245]

docs = tokenizer.make_docs([text1, text2])

## Dealing with Slow Processing Speeds

Part of the reason some language models take a long time is that they have numerous components that handle tasks such as tagging parts of speech, identifying syntactic dependencies, or labelling named entities like people and places. The documentation on the <a href="https://spacy.io/models" target="_blank">spaCy models</a> webpage will identify which components are available in the model's pipeline. If you do not need a component, you can speed up processing times by disabling or excluding components you do not intend to use. Disabled components (listed with the `disable` parameter) will be loaded but unused, and excluded components (listed with the `exclude` parameter) will not be loaded.

Try out the examples below by commenting and uncommenting them to compare how long they take to the process with all pipeline components enabled and included. You won't notice a lot of difference on a short text but you will on a longer one.

In [ ]:
doc = tokenizer.make_doc(text, model="en_core_web_sm", disable=["tagger","parser"])

#doc = tokenizer.make_doc(text, model="en_core_web_sm", exclude=["tagger","parser"])

Tokenisation using spaCy uses a lot of memory. For a small English-language model, the parser and named entity recogniser (NER) can require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or named entity recognition (NER) components, it's probably safe to increase the memory limit with the `max_length` parameter. The limit is in number of characters (the default is set to 2,000,000 for Lexos), so you can check whether your inputs are too long by checking `len(text)`. In some cases, it may be worthwhile cutting texts into segments with `Cutter` and then reassembling them after they have been tokenised.

In [ ]:
print("Character Length of _Pride and Prejudice_:", len(loader.texts[0]))

## Stop Words

A stop word (or "stopword") is a token that you typically wish to remove from your analysis, generally because the token is not a carrier of meaning. Stop words are generally small function words like "and" or "the", but they can also be words like personal names, where the inclusion of those names might skew your data in your intended analysis.

Stop words can be added or removed with `add_stopwords` and `remove_stopwords`. Since the default language model is a multilanguage model, it has no built-in stop words. Models for specific languages generally have built-in stop word lists which you can modify.

In [ ]:
# This import allows us to display multiple dataframes in the same cell
from IPython.display import display

text = "This is an example string to test the tokenizer."

doc1 = tokenizer.make_doc(
    text,
    add_stopwords=["an", "the", "is"]
)
print("\nDefault model with stop words added:")
tokens = [{"Token": token.text, "Stopword": token.is_stop} for token in doc1]
df = pd.DataFrame(tokens)
display(df.head(10))

doc2 = tokenizer.make_doc(
    text,
    model="en_core_web_sm",
    remove_stopwords=["is", "the"]
)
print("\nEnglish language model with stop words removed:")
tokens = [{"Token": token.text, "Stopword": token.is_stop} for token in doc2]
df = pd.DataFrame(tokens)
display(df.head(10))

## Filtering Tokens

You can perform filtering functions based on attributes like `is_stop` and `is_punct`. You can perform similar functions with other token attributes. But be careful. Apart from the `text` attribute and attributes beginning with `is_`, you will want to reference most attributes with a trailing `_` (e.g. `lemma_`, `pos_`). This is a peculiarity of how spaCy references the values of those attributes. If you get a number when trying to get an attribute value, that's how you know you should add `_` to the end of the attribute name. 

In [ ]:
# Use a list comprehension to create a list of filtered tokens forms
filtered_tokens = [token.text for token in doc1 if not token.is_stop and not token.is_punct]

# Convert the list to a space-separated string and make a new doc
filtered_text = " ".join(filtered_tokens)
filtered_doc = tokenizer.make_doc(filtered_text)

# Print the filtered doc
print(filtered_doc.text)

# Print the tokens as a dataframe
tokens = [{"Token": token.text, "Punctuation": token.is_punct, "Stopword": token.is_stop} for token in filtered_doc]
df = pd.DataFrame(tokens)
df

## Generating Word Ngrams

An n-gram is a token consisting of 1 or more units. Each unit is typically a word, but it can also be a character, sentence, or some other countable sequence. A bigram has two units and a trigram has 3 units. Larger ngrams are usually referred to as 4-grams, 5-grams, etc. Analysing n-grams can sometimes be useful because they can capture information such as phrases.

The cell below shows how to ust `tokenizer.ngrams_from_doc()` to convert a document into a list of bigrams.

In [ ]:
text = "This is an example string to test the tokenizer component."
doc = tokenizer.make_doc(text)

ngrams = tokenizer.ngrams_from_doc(doc, size=2)

for ngram in ngrams:
    print(ngram)

Behind the scenes, `tokenizer.ngrams_from_doc()` calls an ngrams function from the `Textacy` text analysis package. Thus we can perform the action above importing and calling Textacy directly. We do this in the example below. Note that we will assign Textacy's `ngrams()` function to the alias `ng` so that we can use the `ngrams` for our list, as above, without a name conflict.

In [ ]:
from textacy.extract.basics import ngrams as ng

text = "This is an example string to test the tokenizer component."
doc = tokenizer.make_doc(text)

ngrams = list(ng(doc, 2, min_freq=1))

for ngram in ngrams:
    print(ngram)

In most cases, it may be simpler to use `tokenizer.ngrams_from_doc()`, but the Textacy method offers some additional options which you can read about in the <a href="https://textacy.readthedocs.io/en/latest/api_reference/extract.html#textacy.extract.basics.ngrams" target="_blank">Textacy documentation</a>.

## Generating Docs From Ngrams

`tokenizer.doc_from_ngrams()` generates a list of ngrams from a spaCy doc. If you want to use the ngrams as a doc you will need to use this function. There is also an equivalent `docs_from_ngrams()` function which you can use to convert multiple lists of ngrams into multiple docs.

Both functions optionally takes the `model`, `disable`, and `exclude` parameters available in `make_doc()`. Additionally, the `strict` parameter shown below allows you to choose whether to preserve token divisions and include whitespace in the source document.

Note that these functions require a list of strings, such as that produced by `tokenizer.ngrams_from_doc()`, as input. If you used Textacy's method, which generates a list of spaCy `span` objects, you must first convert it to a list of strings (see the commented out code in the cell below). 

In [ ]:
# Generate ngrams with Lexos
ngrams = tokenizer.ngrams_from_doc(doc, size=2)

# Generate ngrams with Textacy
# ngrams = list(ng(doc, 2, min_freq=1))
# ngrams = [token.text for token in ngrams]

ngrams_doc = tokenizer.doc_from_ngrams(ngrams, strict=True, model="en_core_web_sm")

for token in ngrams_doc:
    print(token.text)

## Generating Character Ngrams

In the examples above, the text is first tokenised and the tokens are then processed into ngrams. It is also possible to generate ngrams from _untokenised_ text using `tokenizer.generate_character_ngrams()`. This can be useful if you are working in a language like Chinese where words are not separated by spaces or the tokenisation offered by an existing language model is not suitable for your purpose.

The first parameter is the character length of the ngrams and the second is whether white space should be dropped from the resulting list of ngrams.

In [ ]:
text = "This is an example string to test the tokenizer."

ngrams = tokenizer.generate_character_ngrams(text, 2, drop_whitespace=False)
for ngram in ngrams:
    print(ngram)